In [ ]:
import torch
import torch.nn as nn
import numpy as np


class Net(nn.Module):
    def __init__(self, lookup_table,kernel_size, num_filt, num_classes, drop_prob, mode='static'):
        super(Net, self).__init__()
        np.random.seed(99)
        self.mode = mode
        if mode == 'rand':
            ch_num = 1
            new_lookup_table = np.random.uniform(-1,1,lookup_table.shape).astype(np.float32)
            self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(new_lookup_table), freeze=True)
        elif mode == 'static':
            ch_num = 1
            self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(lookup_table), freeze=True)
        elif mode == 'non-static':
            ch_num = 1
            self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(lookup_table), freeze=False)
        elif mode == 'multi-channel':
            ch_num = 2
            self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(lookup_table), freeze=True)
            self.embedding_mult = nn.Embedding.from_pretrained(torch.from_numpy(lookup_table), freeze=False)
        self.convs = nn.ModuleList([nn.Conv2d(ch_num,num_filt,(size,300),1) for size in kernel_size])
        torch.manual_seed(99)
        for conv in self.convs:
            torch.nn.init.kaiming_uniform_(conv.weight)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)
        self.fc = nn.Linear(len(kernel_size)*num_filt, num_classes)
        torch.nn.init.uniform_(self.fc.weight)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, x):
        embed_out = self.embedding(x).unsqueeze(1)
        if self.mode == 'multi-channel':
            embed_mult = self.embedding_mult(x).unsqueeze(1)
            embed_out = torch.cat((embed_out, embed_mult), 1)
        conv_out = [torch.max(self.relu(conv(embed_out)), dim=2)[0].squeeze(-1) for conv in self.convs]
        flatten = torch.cat(conv_out, dim=1)
        dropouts = self.dropout(flatten)
        out = self.fc(dropouts)
        return self.softmax(out)


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time
import nltk
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.model_selection import KFold
import copy
import re
import pickle

w2v = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin', binary=True)

def strCleanup(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

def makeIdx(data, w2i):
    def get_idx(str):
        return w2i[str]
    for idx_sent, sent in enumerate(data):
        for idx, word in enumerate(sent):
            data[idx_sent][idx] = get_idx(word)
#data is tokenized sentences list
def make_w2v(data):
    embedding_dic = {}
    wv_0 = Word2Vec(data, size=300, window=5, min_count=1)
    embedding_arr = np.zeros((len(wv_0.wv.vocab) + 2, 300), dtype=np.float32)
    for idx, (word, vec) in enumerate(zip(wv_0.wv.vocab, wv_0.wv.vectors)):
        coefs = np.asarray(vec, dtype='float32')
        embedding_arr[idx + 2, :] = (coefs / np.linalg.norm(coefs))
        embedding_dic[word] = idx + 2
    return embedding_dic, embedding_arr
#data is set of words
def get_w2v(data):
    global w2v
    tmp_dic = {}
    embedding_dic = {}
    noword = 0
    for word, vec in zip(w2v.vocab, w2v.vectors):
        if word in data:
            tmp_dic[word] = vec
    for word in data:
        if tmp_dic.get(word) is None:
            tmp_dic[word] = np.random.uniform(-1, 1, size=300)
            noword += 1
    embedding_arr = np.zeros((len(tmp_dic) + 2, 300), dtype=np.float32)
    print("PRE words: ", len(data) - noword)
    rand = np.random.uniform(-1, 1, size=300)
    embedding_arr[0] = rand / np.linalg.norm(rand)
    rand = np.random.uniform(-1, 1, size=300)
    embedding_arr[1] = rand / np.linalg.norm(rand)
    for idx, (word, vec) in enumerate(tmp_dic.items()):
        coefs = np.asarray(vec, dtype='float32')
        embedding_arr[idx + 2, :] = (coefs / np.linalg.norm(coefs))
        embedding_dic[word] = idx + 2
    return embedding_dic, embedding_arr

def MR_Preprocess(embed_kinds = 0):
    global w2v
    f_pos = open("/content/drive/MyDrive/Data/rt-polaritydata/rt-polarity.pos", 'r', encoding='latin-1')
    f_neg = open("/content/drive/MyDrive/Data/rt-polaritydata/rt-polarity.neg", 'r', encoding='latin-1')
    lines_pos = f_pos.readlines()
    lines_neg = f_neg.readlines()
    tokenized_pos = [strCleanup(line).split() for line in lines_pos]
    tokenized_neg = [strCleanup(line).split() for line in lines_neg]
    pos_label = [0] * len(tokenized_pos)
    neg_label = [1] * len(tokenized_neg)
    f_pos.close()
    f_neg.close()
    tokenized = tokenized_pos+tokenized_neg
    label = pos_label+neg_label
    tokenized_set = []
    for sent in tokenized:
        tokenized_set.extend(sent)
    tokenized_set = list(set(tokenized_set))
    embedding_dic = {}
    noword = 0

    if not os.path.isfile("/content/drive/MyDrive/Data/rt-polaritydata/MR_dic.pickle"):
        if embed_kinds is 0:
            wv_0 = Word2Vec(tokenized, size=300, window=5, min_count=1)
            embedding_arr = np.zeros((len(wv_0.wv.vocab)+2, 300), dtype=np.float32)
            for idx, (word,vec) in enumerate(zip(wv_0.wv.vocab, wv_0.wv.vectors)):
                coefs = np.asarray(vec, dtype='float32')
                embedding_arr[idx+2,:] = (coefs/np.linalg.norm(coefs))
                embedding_dic[word] = idx+2
        elif embed_kinds is 1:
            tmp_dic = {}
            for word,vec in zip(w2v.vocab, w2v.vectors):
                if word in tokenized_set:
                    tmp_dic[word] = vec
            for word in tokenized_set:
                if tmp_dic.get(word) is None:
                    tmp_dic[word] = np.random.uniform(-1, 1, size=300)
                    noword +=1
            embedding_arr = np.zeros((len(tmp_dic)+2, 300), dtype=np.float32)
            print("PRE words: ",len(tokenized_set) - noword)
            rand = np.random.uniform(-1,1,size = 300)
            embedding_arr[0] = rand/np.linalg.norm(rand)
            rand = np.random.uniform(-1,1,size = 300)
            embedding_arr[1] = rand/np.linalg.norm(rand)
            for idx, (word, vec) in enumerate(tmp_dic.items()):
                coefs = np.asarray(vec, dtype='float32')
                embedding_arr[idx+2,:] = (coefs/np.linalg.norm(coefs))
                embedding_dic[word] = idx+2
        with open('/content/drive/MyDrive/Data/rt-polaritydata/MR_dic.pickle', 'wb') as fw:
            pickle.dump(embedding_dic,fw)
        with open('/content/drive/MyDrive/Data/rt-polaritydata/MR_arr.pickle', 'wb') as fa:
            pickle.dump(embedding_arr,fa)
    else:
        with open('/content/drive/MyDrive/Data/rt-polaritydata/MR_dic.pickle', 'rb') as fw:
            embedding_dic = pickle.load(fw)
        with open('/content/drive/MyDrive/Data/rt-polaritydata/MR_arr.pickle', 'rb') as fa:
            embedding_arr = pickle.load(fa)
    print(embedding_dic)
    makeIdx(tokenized, embedding_dic)
    print(len(tokenized))
    print(len(tokenized[0]))
    tokenized_df = [[tokenized[i], label[i]] for i in range(len(tokenized))]
    print(embedding_arr.shape)
    return tokenized_df, tokenized_set, embedding_arr

def STT_Preprocess(embed_kinds = 0):
    if not os.path.isfile("/content/drive/MyDrive/Data/stanfordSentimentTreebank/SST_Merged.csv"):
        sentences = pd.read_csv("/content/drive/MyDrive/Data/stanfordSentimentTreebank/datasetSentences.txt", sep="\t")
        dic = pd.read_csv("/content/drive/MyDrive/Data/stanfordSentimentTreebank/dictionary.txt", sep="|", names=['phrase', 'phrase ids'])
        labels = pd.read_csv("/content/drive/MyDrive/Data/stanfordSentimentTreebank/sentiment_labels.txt", sep="|")
        split = pd.read_csv("/content/drive/MyDrive/Data/stanfordSentimentTreebank/datasetSplit.txt")
        sentence_phrase = pd.merge(sentences, dic, left_on='sentence', right_on='phrase')
        train_test_split = pd.merge(sentence_phrase, split, on='sentence_index')
        merged = pd.merge(train_test_split, labels, on='phrase ids')
        merged.to_csv("/content/drive/MyDrive/Data/stanfordSentimentTreebank/SST_Merged.csv")
    else:
        merged = pd.read_csv("/content/drive/MyDrive/Data/stanfordSentimentTreebank/SST_Merged.csv")

    def tokenize(str):
        return strCleanup(str).split()
    def check_vocabsize(arr):
        return len(arr)

    merged['sentence'] = merged['sentence'].transform(tokenize)
    vocab_list = []
    for data in merged['sentence']:
        vocab_list.extend(data)
    vocab_list = list(set(vocab_list))
    print('vocab size: ', len(vocab_list))
    def make_label(score):
        if 0 <= score <= 0.2:
            return 0
        elif 0.2 < score <= 0.4:
            return 1
        elif 0.4 < score <= 0.6:
            return 2
        elif 0.6 < score <= 0.8:
            return 3
        elif 0.8 < score <= 1:
            return 4
    merged['label'] = merged['sentiment values'].transform(make_label)
    if not os.path.isfile("/content/drive/MyDrive/Data/stanfordSentimentTreebank/STT_arr.pickle"):
        if embed_kinds is 0:
            embedding_dic, embedding_arr = make_w2v(merged['sentence'].to_numpy())
        elif embed_kinds is 1:
            embedding_dic, embedding_arr = get_w2v(vocab_list)
        with open('/content/drive/MyDrive/Data/stanfordSentimentTreebank/STT_dic.pickle', 'wb') as fw:
            pickle.dump(embedding_dic,fw)
        with open('/content/drive/MyDrive/Data/stanfordSentimentTreebank/STT_arr.pickle', 'wb') as fa:
            pickle.dump(embedding_arr,fa)
    else:
        with open('/content/drive/MyDrive/Data/stanfordSentimentTreebank/STT_dic.pickle', 'rb') as fw:
            embedding_dic = pickle.load(fw)
        with open('/content/drive/MyDrive/Data/stanfordSentimentTreebank/STT_arr.pickle', 'rb') as fa:
            embedding_arr = pickle.load(fa)
    makeIdx(merged['sentence'], embedding_dic)
    train = merged[merged['splitset_label'] == 1]
    test = merged[merged['splitset_label'] == 2]
    dev = merged[merged['splitset_label'] == 3]
    train_df = [[row['sentence'], row['label']] for index,row in train.iterrows()]
    test_df = [[row['sentence'], row['label']] for index,row in test.iterrows()]
    dev_df = [[row['sentence'], row['label']] for index,row in dev.iterrows()]

    return train_df, test_df, dev_df, vocab_list, embedding_arr

def SUBJ_Preprocess(embed_kinds = 0):
    global w2v
    f_sub = open("/content/drive/MyDrive/Data/rotten_imdb.tar/quote.tok.gt9.5000", 'r', encoding='latin-1')
    f_obj = open("/content/drive/MyDrive/Data/rotten_imdb.tar/plot.tok.gt9.5000", 'r', encoding='latin-1')
    lines_sub = f_sub.readlines()
    lines_obj = f_obj.readlines()
    tokenized_sub = [strCleanup(line).split() for line in lines_sub]
    tokenized_obj = [strCleanup(line).split() for line in lines_obj]
    sub_label = [0] * len(tokenized_sub)
    obj_label = [1] * len(tokenized_obj)
    f_sub.close()
    f_obj.close()
    tokenized = tokenized_sub+tokenized_obj
    label = sub_label+obj_label
    tokenized_set = []
    for sent in tokenized:
        tokenized_set.extend(sent)
    tokenized_set = list(set(tokenized_set))
    embedding_dic = {}
    if not os.path.isfile("/content/drive/MyDrive/Data/TREC/TREC_arr.pickle"):
        if embed_kinds is 0:
            embedding_dic, embedding_arr = make_w2v(tokenized)
        elif embed_kinds is 1:
            embedding_dic, embedding_arr = get_w2v(tokenized_set)
        with open('/content/drive/MyDrive/Data/TREC/TREC_dic.pickle', 'wb') as fw:
            pickle.dump(embedding_dic,fw)
        with open('/content/drive/MyDrive/Data/TREC/TREC_arr.pickle', 'wb') as fa:
            pickle.dump(embedding_arr,fa)
    else:
        with open('/content/drive/MyDrive/Data/TREC/TREC_dic.pickle', 'rb') as fw:
            embedding_dic = pickle.load(fw)
        with open('/content/drive/MyDrive/Data/TREC/TREC_arr.pickle', 'rb') as fa:
            embedding_arr = pickle.load(fa)
    makeIdx(tokenized, embedding_dic)
    print(len(tokenized))
    print(len(tokenized[0]))
    tokenized_df = [[tokenized[i], label[i]] for i in range(len(tokenized))]
    print(embedding_arr.shape)
    return tokenized_df, tokenized_set, embedding_arr

def TREC_Preprocess(embed_kinds = 0):
    f_train = open("/content/drive/MyDrive/Data/TREC/train_5500.label.txt", 'r', encoding='latin-1')
    f_test = open("/content/drive/MyDrive/Data/TREC/TREC_10.label.txt", 'r', encoding='latin-1')
    lines_train = f_train.readlines()
    lines_test = f_test.readlines()
    train_label = []
    test_label = []
    train_tokenized = []
    test_tokenized = []

    for line in lines_train:
        line = line.split(":")
        train_label.append(line[0])
        # ? is in every sentence so delete
        line = strCleanup(line[1]).split()[1:-1]
        train_tokenized.append(line)
    for line in lines_test:
        line = line.split(":")
        test_label.append(line[0])
        # ? is in every sentence so delete
        line = strCleanup(line[1]).split()[1:-1]
        test_tokenized.append(line)
    tokenized = train_tokenized+test_tokenized
    f_train.close()
    f_test.close()
    def make_label(cat):
        if cat == 'ABBR':
            return 0
        elif cat == 'ENTY':
            return 1
        elif cat == 'DESC':
            return 2
        elif cat == 'HUM':
            return 3
        elif cat == 'LOC':
            return 4
        elif cat == 'NUM':
            return 5
    label = np.array(train_label+test_label)
    lebelling = np.vectorize(make_label)
    label = lebelling(label)
    tokenized_set = []
    for sent in tokenized:
        tokenized_set.extend(sent)
    tokenized_set = list(set(tokenized_set))
    if not os.path.isfile("/content/drive/MyDrive/Data/TREC/TREC_arr.pickle"):
        if embed_kinds is 0:
            embedding_dic, embedding_arr = make_w2v(tokenized)
        elif embed_kinds is 1:
            embedding_dic, embedding_arr = get_w2v(tokenized_set)
        with open('/content/drive/MyDrive/Data/TREC/TREC_dic.pickle', 'wb') as fw:
            pickle.dump(embedding_dic,fw)
        with open('/content/drive/MyDrive/Data/TREC/TREC_arr.pickle', 'wb') as fa:
            pickle.dump(embedding_arr,fa)
    else:
        with open('/content/drive/MyDrive/Data/TREC/TREC_dic.pickle', 'rb') as fw:
            embedding_dic = pickle.load(fw)
        with open('/content/drive/MyDrive/Data/TREC/TREC_arr.pickle', 'rb') as fa:
            embedding_arr = pickle.load(fa)
    makeIdx(tokenized, embedding_dic)
    train_df = []
    test_df = []
    for idx in range(len(tokenized)):
        if idx < len(train_tokenized):
            train_df.append([tokenized[idx], label[idx]])
        else:
            test_df.append([tokenized[idx], label[idx]])
    return train_df, test_df, tokenized_set, embedding_arr

def CRnMPQA_Preprocess(pos_path, neg_path, embed_kinds = 0, ):
    global w2v
    f_pos = open(pos_path, 'r', encoding='latin-1')
    f_neg = open(neg_path, 'r', encoding='latin-1')
    lines_pos = f_pos.readlines()
    lines_neg = f_neg.readlines()
    tokenized_pos = [strCleanup(line).split() for line in lines_pos]
    tokenized_neg = [strCleanup(line).split() for line in lines_neg]
    pos_label = [0] * len(tokenized_pos)
    neg_label = [1] * len(tokenized_neg)
    f_pos.close()
    f_neg.close()
    tokenized = tokenized_pos+tokenized_neg
    label = pos_label+neg_label
    tokenized_set = []
    for sent in tokenized:
        tokenized_set.extend(sent)
    tokenized_set = list(set(tokenized_set))
    embedding_dic = {}
    if embed_kinds == 0:
        embedding_dic, embedding_arr = make_w2v(tokenized)
    elif embed_kinds == 1:
        embedding_dic, embedding_arr = get_w2v(tokenized_set)
    makeIdx(tokenized, embedding_dic)
    print(len(tokenized))
    print(len(tokenized[0]))
    tokenized_df = [[tokenized[i], label[i]] for i in range(len(tokenized))]
    print(embedding_arr.shape)
    return tokenized_df, tokenized_set, embedding_arr

def batchPad(batch):
    max_len = max([len(sent[0]) for sent in batch])
    if max_len <5:
        max_len = 5
    rows = []
    batch_labels = []
    for i in range(len(batch)):
        rows.append(np.pad(np.array(batch[i][0][:]), (0, max_len), 'constant', constant_values=1)[:max_len])
        batch_labels.append(batch[i][1])
        #print(batch_labels)
    return torch.LongTensor(np.concatenate(rows, axis=0).reshape(-1, max_len)), torch.LongTensor(batch_labels)


#MR_data, MR_wordset, lookup_table = MR_Preprocess(1)
#MR_Train = MR_data[:5000]+MR_data[5332:10333]
#MR_Test = MR_data[5000:5332]+MR_data[10333:]
#np.random.shuffle(MR_Test)
#STT_traindata, STT_testdata, STT_devdata, STT_wordset, lookup_table = STT_Preprocess(1)
SUBJ_data, SUBJ_wordset, lookup_table = SUBJ_Preprocess(1)
SUBJ_Train = SUBJ_data[:4700]+ SUBJ_data[5000:9700]
SUBJ_Test = SUBJ_data[4700:5000]+SUBJ+data[9700:]
#TREC_traindata, TREC_testdata, TREC_wordset, lookup_table = TREC_Preprocess(1)
#CR_data, CR_wordset, lookup_table = CRnMPQA_Preprocess("/content/drive/MyDrive/Data/CR/custrev.pos.txt", "/content/drive/MyDrive/Data/CR/custrev.neg.txt", 1)
#MPQA_data, MPQA_wordset, lookup_table = CRnMPQA_Preprocess("/content/drive/MyDrive/Data/MPQA/mpqa.pos.txt", "/content/drive/MyDrive/Data/MPQA/mpqa.neg.txt", 1)


# Set Model for MR Data
net = Net(lookup_table, [3,4,5], 100, 5, 0.5, mode='multi-channel')

# HyperParameters
dev = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('current device: ', dev)
net.to(dev)
k_folds = 10
num_epochs = 100
batch_size = 50
loss_func = nn.CrossEntropyLoss()
parameters = filter(lambda p: p.requires_grad, net.parameters())
optimizer = optim.Adadelta(parameters, lr = 0.1)

torch.manual_seed(99)
running_loss = 0
correct = 0
total = 0

fold_results = {}
is_kfold = False
if is_kfold:
    kfold = KFold(n_splits=k_folds, shuffle=True)
    for fold, (train, test) in enumerate(kfold.split(SUBJ_Train)):
        train_sampler = torch.utils.data.SubsetRandomSampler(train)
        test_sampler = torch.utils.data.SubsetRandomSampler(test)
        trainloader = torch.utils.data.DataLoader(SUBJ_Train, batch_size=batch_size, sampler=train_sampler, collate_fn=batchPad)
        testloader = torch.utils.data.DataLoader(SUBJ_Train, batch_size=batch_size, sampler=test_sampler, collate_fn=batchPad)
        unseenloader = torch.utils.data.DataLoader(SUBJ_Test, batch_size=batch_size, collate_fn=batchPad)

        for epoch in range(num_epochs):
            running_loss = 0.0
            correct = 0
            total = 0
            for idx,(x, y) in enumerate(iter(trainloader), 0):
                net.train()
                x, y = x.to(dev), y.to(dev)

                out = net(x)
                loss = loss_func(out, y)
                loss.backward()
                nn.utils.clip_grad_norm_(parameters, max_norm=3)
                optimizer.step()
                optimizer.zero_grad()

                running_loss += loss.item()
                if idx % 50 == 49:
                    correct = (torch.max(out, 1)[1] == y).sum().item()
                    total = batch_size
                    print('Training Accuracy: %d ' %(100.0 * correct / total))
                    print('%d/%d' % (correct, total))
                    print('RunningLoss %5d: %.3f' %(idx+1, running_loss))

        correct = 0
        total = 0
        net.eval()
        """
        with torch.no_grad():
            for (x, y)  in iter(testloader):
                x, y = x.to(dev), y.to(dev)
                out = net.forward(x)
                predicted = torch.max(out, 1)[1]
                loss = loss_func(out, y)
                print("test loss: ", loss.item())

                correct += (predicted == y).sum().item()
                total += len(y)
                print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
                print('--------------------------------')
                fold_results[fold] = 100.0 * (correct / total)
        """
        with torch.no_grad():
            for (x, y)  in iter(unseenloader):
                x, y = x.to(dev), y.to(dev)
                out = net.forward(x)
                predicted = torch.max(out, 1)[1]
                loss = loss_func(out, y)
                print("test loss: ", loss.item())

                correct += (predicted == y).sum().item()
                total += len(y)
                print('Accuracy for fold %d: %.3f %%' % (fold, 100.0 * correct / total))
                print('--------------------------------')
                fold_results[fold] = 100.0 * (correct / total)

    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in fold_results.items():
        print(f'Fold {key}: {value} %')
        sum += value
        print(f'Average: {sum/len(fold_results.items())} %')
else:
    trainloader = torch.utils.data.DataLoader(STT_traindata, batch_size=batch_size, shuffle=True, collate_fn=batchPad)
    testloader = torch.utils.data.DataLoader(STT_testdata, batch_size=batch_size, shuffle= True, collate_fn=batchPad)

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for idx, (x, y) in enumerate(iter(trainloader), 0):
            net.train()
            x, y = x.to(dev), y.to(dev)

            out = net(x)
            loss = loss_func(out, y)
            loss.backward()
            nn.utils.clip_grad_norm_(parameters, max_norm=3)
            optimizer.step()
            optimizer.zero_grad()

            running_loss += loss.item()
            if idx % 50 == 49:
                correct = (torch.max(out, 1)[1] == y).sum().item()
                total = batch_size
                print('Training Accuracy: %d ' % (100.0 * correct / total))
                print('%d/%d' % (correct, total))
                print('RunningLoss %5d: %.3f' % (idx + 1, running_loss))

    correct_eachbatch = 0
    total_eachbatch = 0
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for (x, y) in iter(testloader):
            x, y = x.to(dev), y.to(dev)
            out = net.forward(x)
            predicted = torch.max(out, 1)[1]
            loss = loss_func(out, y)
            print("test loss: ", loss.item())
            correct_eachbatch += (predicted == y).sum().item()
            correct += correct_eachbatch
            total_eachbatch += len(y)
            total += total_eachbatch
            correct_eachbatch = 0
            total_eachbatch = 0
            print('Accuracy for test_batch: %.3f %%' % (100.0 * correct / total))
            print('--------------------------------')

print('Accuracy for test: %.3f %%' % (100.0 * correct / total))
print('--------------------------------')